# **Industrialisation d'API avec FastAPI & Google Cloud Platform (GCP)**

## **Objectif du Notebook**
Ce module vise à vous apprendre à exposer vos modèles d'IA via des points de terminaison (endpoints) professionnels. Nous allons quitter l'environnement de tracking pour créer une API robuste avec **FastAPI**, la conteneuriser, et la déployer à grande échelle sur l'infrastructure de **Google Cloud**.



# **Partie 1 : Développement d'API avec FastAPI**

**FastAPI** est actuellement le framework Python de référence pour créer des APIs performantes. Il est basé sur des standards modernes (Python 3.6+ type hints) et offre une documentation interactive automatique (Swagger).

In [ ]:
# Installation
!pip install fastapi uvicorn mlflow

In [ ]:
# Fonctionnement de l'API
import fastapi, uvicorn

app = fastapi.FastAPI()

# Création de la route princpal
@app.get("/")
def say_hello():
    return {"message": "Hello, World!"}


# Création d'une route pour calculer le carré d'un nombre
@app.get("/square")
def square_number(number: int):
    return {"result": number ** 2}


# Création d'une route pour recevoir des données JSON
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    value: int

@app.post("/data")
def receive_data(item: Item):
    return {"message": f"Received item with name: {item.name} and value: {item.value}"}


# Envoi de fichiers
from fastapi import UploadFile, File

@app.post("/upload")
def upload_file(file: UploadFile = File(...)):
    content = file.file.read()
    return {"filename": file.filename}


# Lancement de l'API
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
# Lancement
!uvicorn main:app --reload

In [ ]:
# Test de l'API
import requests


# Test de la route principale
response = requests.get("http://localhost:8000/")
print(response.json())  # Affiche: {'message': 'Hello, World!'}


# Test de la route de calcul du carré
response = requests.get("http://localhost:8000/square", params={"number": 5})
print(response.json())  # Affiche: {'result': 25}

# Test de la route de réception de données JSON
response = requests.post("http://localhost:8000/data", json={"name": "Test Item", "value": 42})
print(response.json())  # Affiche: {'message': 'Received item with name: Test Item and value: 42'}

# Test de la route d'envoi de fichiers
with open("test_file.txt", "rb") as f:
    response = requests.post("http://localhost:8000/upload", files={"file": f})


In [1]:
import requests


# Test de la route principale
response = requests.get("http://localhost:8000/")
print(response.json())  # Affiche: {'message': 'Hello, World!'}

{'message': 'Hello, World!'}


In [4]:
response = requests.get("http://localhost:8000/square", params={"number": 5})
print(response.json())  # Affiche: {'result': 25}

{'result': 25}


In [7]:
requests.post("http://localhost:8000/user_infos", json={"name": "Test Item", "age": 25, "email": "test@example.com"}).json()

{'message': 'User Test Item created successfully!'}

In [9]:
with open("main.py", "rb") as f:
    response = requests.post("http://localhost:8000/upload", files={"file": f})

FileNotFoundError: [Errno 2] No such file or directory: 'main.py'

### Projet : L'API d'Inférence Industrielle
Votre mission est de créer un backend capable d'interagir avec votre serveur MLflow distant.

**🛠️ Tâches à réaliser :**

1.  **Point de terminaison `/generate` (POST) :**
    *   Il doit appeler le modèle **GPT-2** hébergé sur votre instance MLflow.
    *   **Paramètres :** `prompt` (string), `max_tokens` (int).
    *   **Action :** Utiliser `mlflow.pyfunc.load_model` ou l'API REST de MLflow pour générer du texte.
    *   **Lien du modèle :** [GPT-2 sur MLflow Azure](https://mlflow-ynov-e9d6hhb5hjhkargh.francecentral-01.azurewebsites.net/#/models/GPT-2-Text-Generation)

2.  **Point de terminaison `/load_model` (GET) :**
    *   Il doit permettre de télécharger physiquement un modèle depuis le registre MLflow vers le stockage local de l'API.
    *   **Paramètres :** `model_name` (string), `version` (int).
    *   **Action :** Utiliser `mlflow.artifacts.download_artifacts`.

3.  **Conteneurisation :**
    *   Rédiger un `Dockerfile` optimisé pour FastAPI.
    *   Tester l'image localement : `docker run -p 8000:8000 pseudo/api-mlflow`.

In [12]:
import os 
import mlflow

os.environ["AZURE_STORAGE_CONNECTION_STRING"] = "DefaultEndpointsProtocol=https;AccountName=queraartifactsstorage;AccountKey=rkbQJGRGAmzMGcYD9kKCJya3FaITjEUjIohybDMfui707ywCw31Hk8u3l/VH8tWq3X5jn05ViYZk+ASttrSL6g==;EndpointSuffix=core.windows.net"
client = mlflow.tracking.MlflowClient("https://mlflow-ynov-e9d6hhb5hjhkargh.francecentral-01.azurewebsites.net/")

run_id = client.get_registered_model("GPT-2-Text-Generation")
client.download_artifacts(run_id, "model", dst_path="./downloaded_model")

TypeError: 'NoneType' object is not iterable

TypeError: 'NoneType' object is not iterable

# **Partie 2 : Google Cloud Platform (GCP)**

GCP est réputé pour son excellence en matière de gestion de conteneurs et d'outils d'IA (Vertex AI).

### 2.1 Les Services Incontournables
*   **[Cloud Run](https://console.cloud.google.com/run) :** Déploiement de conteneurs serverless (idéal pour nos APIs).
*   **[Vertex AI](https://console.cloud.google.com/vertex-ai) :** La plateforme IA unifiée de Google.
*   **[Artifact Registry](https://cloud.google.com/artifact-registry) :** Pour stocker vos images Docker privées.
*   **[Compute Engine (GCE)](https://console.cloud.google.com/compute) :** Machines virtuelles haute performance (VMs).
*   **[Cloud Storage](https://console.cloud.google.com/storage) :** Stockage objet (équivalent au Blob Azure).

### 2.2 Configuration & CLI
Téléchargez et installez le **[Google Cloud SDK](https://cloud.google.com/sdk/docs/install)**.
Identifiez-vous : `gcloud auth login`

### 2.3 Déploiement : Le Workflow Artifact Registry
Nous allons utiliser le projet Google nommé `925284333159` pour héberger nos images.

**Commandes de publication :**

In [ ]:
# 1. Configurer l'authentification Docker pour Google
gcloud auth configure-docker europe-west1-docker.pkg.dev

# 2. Créer le dépôt (si non existant)
gcloud artifacts repositories create api-mlflow --project=925284333159 --repository-format=docker --location=europe-west1

# 3. Taguer votre image locale vers le registre Google
docker tag pseudo/api-mlflow europe-west1-docker.pkg.dev/925284333159/api-mlflow/api-mlflow

# 4. Pusher l'image sur Google Cloud
docker push europe-west1-docker.pkg.dev/925284333159/api-mlflow/api-mlflow

# 🚀 Projets de Déploiement

### Projet 1 : Déploiement de l'API (Backend)
Déployez votre conteneur `api-mlflow` sur **Cloud Run**.
*   Configurez les variables d'environnement pour que l'API puisse joindre le serveur MLflow Azure.
*   Validez le fonctionnement via le Swagger auto-généré (`/docs`).

### Projet 2 : Déploiement du Front (Interface)
Déployez une application **Streamlit** (sur Cloud Run également) qui servira d'interface utilisateur.
*   L'utilisateur saisit un prompt dans Streamlit.
*   Streamlit envoie une requête `POST` à votre API sur Cloud Run.
*   L'IA génère le texte et l'affiche à l'écran.

---

### Projet 3 : Calcul Distribué avec Google Compute (GCE)
Nous allons utiliser la puissance de calcul brute de Google pour nos entraînements lourds.
*   Déployez votre image d'entraînement "Cats vs Dogs" (vue au Notebook 2) sur une instance **Google Compute Engine**.
*   Vérifiez que l'instance est capable de tirer les données de GitHub et de loguer les résultats sur votre MLflow central.


### Projet 4 : Computer Vision
Nous allons intégrer le modèle de computer vision dans un point de terminaison de notre api et ajouter une page à notre application front pour permettre l'inférence du modèle.
*   Créez un point de terminaison `computer_vision` qui import le modèle depuis MLFlow fait une prédiction.
*   Ajoutez une page 'COmputer Vision' à votre application front.